## Various Imports

In [1]:
import os
import sys
import shutil
import numpy as np
from collections import Counter

# third-party packages
import pandas as pd
import numpy as np
from PIL import Image
from scipy.spatial import distance

import matplotlib.pyplot as plt 
%matplotlib inline

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import backend as K

print(f"Current data format {K.image_data_format()}")
K.set_image_data_format('channels_last')
print(f"New data format {K.image_data_format()}")

from converter.xml_analyzer import parse_xml
from converter.xml_analyzer import get_conv_weights
from converter.xml_analyzer import get_affine_weights
from converter.xml_analyzer import count_number_of_layers

Current data format channels_last
New data format channels_last


In [2]:
# You may need to adjust these paths if you
# have not followed the instructions in README.md
# and have generated the files at different locations

REPO_PATH = os.curdir

# Adjust these as per your machines
PATH_TO_DLIB_XML_MODEL = os.path.join(REPO_PATH, 'xml_generator', "build/dlib_face_recognition_resnet_model_v1.xml")
PATH_TO_DLIB_H5_MODEL = os.path.join(REPO_PATH, "dlib_face_recognition_resnet_model_v1.h5")

TEST_IMAGE_FOLDER = os.path.join(REPO_PATH, "images")

# note - these images are already aligned using dlib and of size (150,150)
TEST_IMAGES = [
    "0001_01.png",  # class 1
    "0010_01.png",  # class 1
    "0096_01.png",  # class 2
    "0096_02.png",  # class 2
    "0096_04.png"   # class 2
]

PATH_TO_ALL_TEST_IMAGES = list(map(lambda x: os.path.join(TEST_IMAGE_FOLDER, x), TEST_IMAGES))

print(f"{TEST_IMAGE_FOLDER}")
print(f"{PATH_TO_DLIB_XML_MODEL}")
print(f"{PATH_TO_DLIB_H5_MODEL}")

# sanity tests that these paths do exist
assert os.path.exists(TEST_IMAGE_FOLDER)
assert os.path.exists(PATH_TO_DLIB_XML_MODEL)
assert os.path.exists(PATH_TO_DLIB_H5_MODEL)

./images
./xml_generator/build/dlib_face_recognition_resnet_model_v1.xml
./dlib_face_recognition_resnet_model_v1.h5


## Dlib's Model Analysis

In [3]:
# parse it
xml_model = parse_xml(PATH_TO_DLIB_XML_MODEL)

In [4]:
ln_vs_count = count_number_of_layers(xml_model)
conv_layers_info = get_conv_weights(xml_model)[::-1]
affine_layers_info = get_affine_weights(xml_model)[::-1]

Total number of layer entries - 132


In [5]:
# let pandas do it magic 
pd.DataFrame.from_dict(ln_vs_count, orient='index', columns=['Count'])

Count
loss_metric          1
fc_no_bias           1
avg_pool             5
relu                29
add_prev            14
affine_con          29
con                 29
max_pool             1
input_rgb_image      1

In [6]:
# Conv layers
pd.DataFrame(conv_layers_info)

id     name  nc  nr  num_filters  sx  sy  total_weights  \
0   130   conv_1   7   7           32   2   2           4736   
1   125   conv_2   3   3           32   1   1           9248   
2   122   conv_3   3   3           32   1   1           9248   
3   117   conv_4   3   3           32   1   1           9248   
4   114   conv_5   3   3           32   1   1           9248   
5   109   conv_6   3   3           32   1   1           9248   
6   106   conv_7   3   3           32   1   1           9248   
7   101   conv_8   3   3           64   2   2          18496   
8    98   conv_9   3   3           64   1   1          36928   
9    90  conv_10   3   3           64   1   1          36928   
10   87  conv_11   3   3           64   1   1          36928   
11   82  conv_12   3   3           64   1   1          36928   
12   79  conv_13   3   3           64   1   1          36928   
13   74  conv_14   3   3           64   1   1          36928   
14   71  conv_15   3   3           64   1   1          36928   
15   66  conv_16   3   3          128   2   2          73856   
16   63  conv_17   3   3          128   1   1         147584   
17   55  conv_18   3   3          128   1   1         147584   
18   52  conv_19   3   3          128   1   1         147584   
19   47  conv_20   3   3          128   1   1         147584   
20   44  conv_21   3   3          128   1   1         147584   
21   39  conv_22   3   3          256   2   2         295168   
22   36  conv_23   3   3          256   1   1         590080   
23   28  conv_24   3   3          256   1   1         590080   
24   25  conv_25   3   3          256   1   1         590080   
25   20  conv_26   3   3          256   1   1         590080   
26   17  conv_27   3   3          256   1   1         590080   
27   12  conv_28   3   3          256   2   2         590080   
28    9  conv_29   3   3          256   1   1         590080   

                                              weights  
0   [-0.0293299202, -0.0188653059, -0.00566938985,...  
1   [-0.0198807381, -0.0278012436, 0.00704712234, ...  
2   [2.06610493e-05, -0.00611053547, -0.0030803154...  
3   [-0.0184960347, 0.00806789286, 0.00271707121, ...  
4   [-0.00948836561, -0.00907282066, 0.0125354007,...  
5   [0.00616474077, 0.00430707354, -0.0156090874, ...  
6   [0.00438096095, -0.0100570731, -0.00921011623,...  
7   [-0.00449227169, 0.000273505109, 0.0122217648,...  
8   [-0.00792279281, -0.00957604777, -0.0077794729...  
9   [-0.0032715241, -0.0105726216, -0.0125644859, ...  
10  [-0.00599347986, 0.00693058176, -0.0027659561,...  
11  [0.00780967856, 0.00809686352, -0.0147641767, ...  
12  [-0.00470875483, -0.00185131689, -0.0046570436...  
13  [-0.00879808981, 0.00467669452, -0.00511790346...  
14  [0.00261793821, 0.017181132, 0.00580968754, 0....  
15  [-0.000180202915, -0.00242396048, -0.003859455...  
16  [0.00157160102, -0.00199561892, -0.00593280001...  
17  [-0.00497292168, 0.002867084, -0.00429081731, ...  
18  [0.00646644505, 0.00128645333, 0.00617424725, ...  
19  [-0.000916853722, 0.00136676151, -0.0009121019...  
20  [-0.00858977996, -0.00547988294, 0.00292515452...  
21  [0.00115965947, 0.00308754179, 0.00197676569, ...  
22  [-0.00321936561, -0.00203424762, 4.19996213e-0...  
23  [-0.000665110594, -0.00139071338, -3.52550269e...  
24  [-0.000944898697, 0.000316396123, -0.000241531...  
25  [-0.000134924441, -6.64100435e-06, 7.18580268e...  
26  [1.30998274e-07, 1.39638018e-06, 1.67919679e-0...  
27  [-0.00466986047, 0.000429832347, 0.00492245844...  
28  [3.87452959e-09, 4.33794378e-09, 2.22511565e-0...

In [7]:
# Affine (scale) layers
pd.DataFrame(affine_layers_info)

id   name  total_weights  \
0   129   sc_1             64   
1   124   sc_2             64   
2   121   sc_3             64   
3   116   sc_4             64   
4   113   sc_5             64   
5   108   sc_6             64   
6   105   sc_7             64   
7   100   sc_8            128   
8    97   sc_9            128   
9    89  sc_10            128   
10   86  sc_11            128   
11   81  sc_12            128   
12   78  sc_13            128   
13   73  sc_14            128   
14   70  sc_15            128   
15   65  sc_16            256   
16   62  sc_17            256   
17   54  sc_18            256   
18   51  sc_19            256   
19   46  sc_20            256   
20   43  sc_21            256   
21   38  sc_22            512   
22   35  sc_23            512   
23   27  sc_24            512   
24   24  sc_25            512   
25   19  sc_26            512   
26   16  sc_27            512   
27   11  sc_28            512   
28    8  sc_29            512   

                                              weights  
0   [12.1663227, 10.5592213, 17.4300499, 13.288585...  
1   [5.95501137, 5.37066984, 4.13563061, 6.4691848...  
2   [5.16520643, 10.5300217, 10.1555595, 9.2638483...  
3   [4.30762815, 4.81665754, 4.58528137, 4.8235268...  
4   [7.71606302, 13.1466589, 14.3001995, 10.426322...  
5   [3.27119374, 4.18873596, 3.24244237, 3.3145091...  
6   [8.37581635, 13.5131445, 17.2815819, 10.821703...  
7   [3.4770906, 3.43965483, 3.10719991, 3.43313265...  
8   [5.06062937, 6.16682053, 8.23571873, 4.0493001...  
9   [4.46614695, 4.8219943, 4.14754963, 3.99555492...  
10  [6.70087481, 8.50363159, 11.2660618, 5.7611289...  
11  [3.43156505, 4.34112453, 4.01937819, 3.6848521...  
12  [9.93892384, 11.882947, 13.2439852, 6.76520538...  
13  [3.90004158, 3.16286612, 2.95372725, 3.7937145...  
14  [10.2004566, 11.3748789, 12.0578995, 8.3372764...  
15  [4.73023415, 3.75052524, 3.64424872, 4.6053214...  
16  [4.14012337, 4.45800495, 3.75378227, 3.6356318...  
17  [6.57886076, 6.48051071, 5.84152412, 5.5515756...  
18  [4.91507053, 5.448452, 4.51402235, 5.18087435,...  
19  [5.78292465, 5.42327261, 5.69609165, 5.5002374...  
20  [5.39306879, 5.87008762, 5.49020147, 5.5736498...  
21  [5.62751341, 6.92813492, 7.74335051, 6.0825266...  
22  [6.45476294, 6.48839331, 6.06291485, 5.8829813...  
23  [9.33516407, 9.13299465, 7.21495914, 12.240094...  
24  [14.9008808, 6.87232161, 13.0418358, 14.295013...  
25  [11.4107761, 8.56124687, 13.0448284, 12.115539...  
26  [5.53916359, 5.69926739, 5.75786781, 10.487877...  
27  [5.73154974, 5.54872274, 5.33928156, 5.7733650...  
28  [12.6955128, 9.89618778, 9.60454655, 7.1134357...

## Probe image preparation

In [8]:
# Various utilities methods to load and pre-process images
def load_image(infilename) :
    img = Image.open(infilename )
    return img

def show_images(images, cols = 1):
    n_images = len(images)
    titles = ['Image (%d)' % i for i in range(1,n_images + 1)]
    fig = plt.figure()
    for n, (image, title) in enumerate(zip(images, titles)):
        a = fig.add_subplot(cols, np.ceil(n_images/float(cols)), n + 1)
        plt.imshow(image)
        a.set_title(title)
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_images)
    plt.show()
    
def convert_image_to_np(img):
    return np.asarray(img, dtype='float32')

def normalize_image(image):    
    [R,G,B] = np.dsplit(image,image.shape[-1])

    Rx = (R - 122.782) / 256.
    Gx = (G - 117.001) / 256.
    Bx = (B - 104.298) / 256.

    new_image = np.dstack((Rx,Gx,Bx))
    return new_image

In [9]:
_test_images = list(map(load_image, PATH_TO_ALL_TEST_IMAGES))
show_images(_test_images, cols=1)

In [10]:
# convert the images to nump arrays
probe_images = list(map(convert_image_to_np, _test_images))

In [11]:
# normalize the images
normalized_probe_images = list(map(normalize_image, probe_images))
normalized_probe_images[0].shape

(150, 150, 3)

In [12]:
reshaped_probe_images = list(map(lambda x: np.reshape(x, (1,150,150,3)), normalized_probe_images))

## Inference using Keras Model

In [13]:
# first we need to load it
# now since we have custom layers we need to specify them

from converter.model import ScaleLayer, ReshapeLayer

model_from_file = tf.keras.models.load_model(PATH_TO_DLIB_H5_MODEL,
    custom_objects={'ScaleLayer': ScaleLayer, 'ReshapeLayer': ReshapeLayer})

model_from_file.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 72, 72, 32)   4736        input_image[0][0]                
__________________________________________________________________________________________________
sc_1 (ScaleLayer)               (None, 72, 72, 32)   64          conv_1[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 72, 72, 32)   0           sc_1[0][0]                       
__________________________________________________________________________________________________
max_poolin

### Moment of Truth :)

There are 5 test (probe) images, the first 2 belongs to class 1 and rest belongs to class 2.

The third image of the class 2 is a pose image and bit blurry as well so I do expect to see the distance between this 
image and the rest of the images to be larger than 0.6 for class 2.

In [15]:
# run the prediction on the normalized images
outputs = list(map(lambda x: model_from_file.predict(x, batch_size=1), reshaped_probe_images))

# the predict returns an array of predictions
# so here we are further destructuring it
(output1, output2, output3, output4, output5) = list(map(lambda x: x[0], outputs))


print(f"image1vs2 {distance.euclidean(output1, output2)}")
print(f"image2vs3 {distance.euclidean(output2, output3)}")
print(f"image3vs4 {distance.euclidean(output3, output4)}")
print(f"image3vs5 {distance.euclidean(output3, output5)}")
print(f"image4vs5 {distance.euclidean(output4, output5)}")

image1vs2 0.33883780241012573
image2vs3 0.9505220055580139
image3vs4 0.39037615060806274
image3vs5 0.522692859172821
image4vs5 0.5595120191574097
